# Preprocessing TLC data
This notebook cleans the following datasets: 
1. Yellow taxi data from 2023-12 to 2024-05
2. Green taxi data from 2023-12 to 2024-05


In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium

In [2]:
from pyspark.sql import SparkSession

# Create a spark session with increased memory allocation
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "8g")  # Set the driver memory to 8GB
    .config("spark.executor.memory", "8g")  # Set the executor memory to 8GB
    .getOrCreate()
)


24/08/27 18:08:19 WARN Utils: Your hostname, Hanshis-Laptop.local resolves to a loopback address: 127.0.0.1; using 10.12.200.32 instead (on interface en0)
24/08/27 18:08:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/27 18:08:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.conf.set("spark.sql.parquet.compression.codec","gzip")

In [4]:
# Read 2023-2024 TLC data
df = spark.read.parquet('../data/landing/tlc_data/*.parquet')


In [5]:
# Read 2023-6 to 2024-5 yellow data
path = "../data/landing/tlc_data"
ydf_2023_12 = spark.read.parquet(path + "/Y-2023-12.parquet")
ydf_2024_1 = spark.read.parquet(path + "/Y-2024-01.parquet")
ydf_2024_2 = spark.read.parquet(path + "/Y-2024-02.parquet")
ydf_2024_3 = spark.read.parquet(path + "/Y-2024-03.parquet")
ydf_2024_4 = spark.read.parquet(path + "/Y-2024-04.parquet")
ydf_2024_5 = spark.read.parquet(path + "/Y-2024-05.parquet")

In [6]:
# Read 2023-12 to 2024-5 green data
path = "../data/landing/tlc_data"
gdf_2023_12 = spark.read.parquet(path + "/G-2023-12.parquet")
gdf_2024_1 = spark.read.parquet(path + "/G-2024-01.parquet")
gdf_2024_2 = spark.read.parquet(path + "/G-2024-02.parquet")
gdf_2024_3 = spark.read.parquet(path + "/G-2024-03.parquet")
gdf_2024_4 = spark.read.parquet(path + "/G-2024-04.parquet")
gdf_2024_5 = spark.read.parquet(path + "/G-2024-05.parquet")

## Data inspection

In [7]:
# Calculate the total row count for yellow taxi data from 2023-6 to 2024-5
yellow_count = (
    ydf_2023_12.count() + 
    ydf_2024_1.count() + 
    ydf_2024_2.count() + 
    ydf_2024_3.count() + 
    ydf_2024_4.count() + 
    ydf_2024_5.count()
)

# Calculate the total row count for green taxi data from 2023-6 to 2024-5
green_count = (
    gdf_2023_12.count() + 
    gdf_2024_1.count() + 
    gdf_2024_2.count() + 
    gdf_2024_3.count() + 
    gdf_2024_4.count() + 
    gdf_2024_5.count()
)

# Display the green count
print(f"The total green count is {green_count}.")

# Display the yellow count
print(f"The total yellow count is {yellow_count}.")

# Calculate the total row count for all taxi data from 2023-6 to 2024-5
total_count = yellow_count + green_count
# Display the total count
print(f"The total count is {total_count}.")

The total green count is 349274.
The total yellow count is 20169467.
The total count is 20518741.


In [8]:
# Get columns of each DataFrame
columns_ydf = set(ydf_2024_5.columns)
columns_gdf = set(gdf_2024_5.columns)

# Find differences in columns
columns_only_in_df1 = columns_ydf - columns_gdf
columns_only_in_df2 = columns_gdf - columns_ydf

print(f"Columns only in yellowDF: {columns_only_in_df1}")
print(f"Columns only in greenDF: {columns_only_in_df2}")


Columns only in yellowDF: {'Airport_fee', 'tpep_pickup_datetime', 'tpep_dropoff_datetime'}
Columns only in greenDF: {'lpep_pickup_datetime', 'ehail_fee', 'trip_type', 'lpep_dropoff_datetime'}


In [9]:
# Report number of features in each DataFrame
print(f"Number of features in yellowDF: {len(ydf_2024_5.columns)}")
print(f"Number of features in greenDF: {len(gdf_2024_5.columns)}")

Number of features in yellowDF: 19
Number of features in greenDF: 20


## Data Cleaning

In [10]:
# combine the datasets 
ydfs = [ydf_2023_12, 
       ydf_2024_1, ydf_2024_2, ydf_2024_3, ydf_2024_4, ydf_2024_5]
gdfs = [gdf_2023_12, 
       gdf_2024_1, gdf_2024_2, gdf_2024_3, gdf_2024_4, gdf_2024_5]


In [11]:
from pyspark.sql import DataFrame
from functools import reduce

# Function to union two DataFrames
def union_dfs(df1, df2):
    return df1.unionByName(df2)

# Combine all yellow taxi data
yellow_combined = reduce(union_dfs, ydfs)

# Combine all green taxi data
green_combined = reduce(union_dfs, gdfs) 

### Unify the columns of the two dataframes

In [12]:
# Drop Airport_fee and trip_type columns from the combined data
yellow_combined = yellow_combined.drop("Airport_fee")
green_combined = green_combined.drop("trip_type")

# Set ehail_fee to 0 for yellow taxi data
yellow_combined = yellow_combined.withColumn("ehail_fee", lit(0))

tpep: Taxicab Passenger Enhancement Program for yellow taxi <br> 
lpep: Livery Passenger Enhancement Program for green taxi

In [13]:
# Rename datetime columns to be consistent
from pyspark.sql.functions import col

yellow_combined = yellow_combined.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
                               .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

green_combined = green_combined.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
                             .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')


### Combine yellow and green taxi data

In [14]:
# Combine yellow and green taxi data
combined = yellow_combined.unionByName(green_combined)

### Anomaly handling 1
Filter out anomaly with business logic

In [15]:
# print the schema of the combined data
combined.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- ehail_fee: double (nullable = true)



In [16]:
from pyspark.sql.functions import min, max

columns_to_check = [
    'passenger_count', 
    'trip_distance', 
    'fare_amount', 
    'extra', 
    'mta_tax', 
    'tip_amount', 
    'tolls_amount', 
    'improvement_surcharge',
    'total_amount',
    'congestion_surcharge',
    'ehail_fee'
]

# Create a dictionary to store min and max for each column
min_max_dict = {col: df.agg(min(col).alias(f"min_{col}"), max(col).alias(f"max_{col}")).collect()[0] for col in columns_to_check}

# Print the results
for col, values in min_max_dict.items():
    print(f"{col}: Min = {values[f'min_{col}']}, Max = {values[f'max_{col}']}")


CodeCache: size=131072Kb used=33039Kb max_used=33071Kb free=98032Kb
 bounds [0x00000001069e8000, 0x0000000108a68000, 0x000000010e9e8000]
 total_blobs=12740 nmethods=11756 adapters=896
 compilation: disabled (not enough contiguous free space left)
passenger_count: Min = 0, Max = 9
trip_distance: Min = 0.0, Max = 345729.44
fare_amount: Min = -1087.3, Max = 386983.63
extra: Min = -39.17, Max = 10002.5
mta_tax: Min = -0.5, Max = 52.09
tip_amount: Min = -330.88, Max = 4174.0
tolls_amount: Min = -91.3, Max = 1702.88
improvement_surcharge: Min = -1.0, Max = 1.0
total_amount: Min = -1094.05, Max = 386987.63
congestion_surcharge: Min = -2.75, Max = 2.75
ehail_fee: Min = None, Max = None


Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


In [17]:
from pyspark.sql.functions import col

# Apply all filters in a single chain
combined = combined.filter(
    # Filter out rows with passenger count greater than 6 or less than 1
    (col("passenger_count").between(1, 6)) &
    # Filter out rows with fare amount less than 3
    (col("fare_amount") >= 3) &
    # Filter out rows with trip distance less than 0.5 miles 
    (col("trip_distance") >= 0.5) &
    # Filter out rows with tip amount less than 0 
    (col("tip_amount") >= 0) &
    # Filter out rows with tolls amount less than 0
    (col("tolls_amount") >= 0) &
    # Filter out rows with extra amount less than 0
    (col("extra") >= 0) &
    # Filter out mtax_tax less than 0
    (col("mta_tax") >= 0) &
    # Filter out rows with improvement surcharge less than 0
    (col("improvement_surcharge") >= 0) &
    # Filter out rows with total amount less than 3
    (col("total_amount") >= 3) &
    # Filter out rows with congestion surcharge less than 0
    (col("congestion_surcharge") >= 0) &
    # Filter the pick up datetime to between 2023-06 to 2024-05
    (col("pickup_datetime").between("2023-06-01 00:00:00", "2024-05-31 00:00:00")) &
    # Filter the drop off datetime to between 2023-06 to 2024-05
    (col("dropoff_datetime").between("2023-06-01 00:00:00", "2024-05-31 00:00:00"))

)


### Data type conversion

In [18]:
# Convert 'store_and_fwd_flag' to boolean
combined = combined.withColumn("store_and_fwd_flag", when(col("store_and_fwd_flag") == "Y", True).otherwise(False))

In [19]:
# Step 1: Convert pickup and dropoff timestamps to string format
combined = combined.withColumn("pickup_str", F.col("pickup_datetime").cast("string"))
combined = combined.withColumn("dropoff_str", F.col("dropoff_datetime").cast("string"))

# Step 2: Convert the strings back to timestamps
combined = combined.withColumn("pickup_ts", F.to_timestamp("pickup_str"))
combined = combined.withColumn("dropoff_ts", F.to_timestamp("dropoff_str"))

# Step 3: Convert the timestamps to long (Unix epoch seconds)
combined = combined.withColumn("pickup_long", F.col("pickup_ts").cast("long"))
combined = combined.withColumn("dropoff_long", F.col("dropoff_ts").cast("long"))

# Step 4: Calculate the trip duration in minutes
combined = combined.withColumn("trip_duration", F.round((F.col("dropoff_long") - F.col("pickup_long")) / 60))

# Drop intermediate columns if no longer needed
combined = combined.drop("pickup_str", "dropoff_str", "pickup_ts", "dropoff_ts", "pickup_long", "dropoff_long")

# Show the result
combined.show(5)


+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+---------+-------------+
|VendorID|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|ehail_fee|trip_duration|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+---------+-------------+
|       1|2023-12-01 00:59:44|2023-12-01 01:13:22|              2|          2.2|         1|             false|         114|         186|           1|       13.5

In [20]:
# check datatypes
combined.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = false)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- trip_duration: double (nullable = true)



From Northern end of bronx to southern end of staten island is below 50 miles </br>
2 hours should be sufficient for travel 

In [21]:
from pyspark.sql.functions import col as pyspark_col

# Filter out rows with trip duration greater than 120 minutes
combined = combined.filter(
    (pyspark_col("trip_duration") <= 120)
)

# Filter out rows with trip distance greater than 50 miles 
combined = combined.filter(
    pyspark_col("trip_distance") <= 50
) 

### Handling missing values & duplicates

In [22]:
# Drop dulicate rows
combined = combined.dropDuplicates()

In [23]:
# Drop rows with missing values
combined = combined.dropna() 

### Anomaly handling by statistics

In [24]:
# keep 99% quantile of total_amount
total_amount_quantile = combined.approxQuantile("total_amount", [0.99], 0.01)[0]
combined = combined.filter(pyspark_col("total_amount") <= total_amount_quantile)

### Data cleaning outcomes

In [55]:
# check the percentage of rows that remain after filtering
original_count = total_count
filtered_count = combined.count()
percentage_remaining = (filtered_count / original_count) * 100
print(f"Percentage of rows remaining after filtering: {percentage_remaining:.2f}%")

Percentage of rows remaining after filtering: 82.31%


In [26]:
# show schema
combined.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = false)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- trip_duration: double (nullable = true)



## Feature Engineering

### Temporal Features

In [27]:
# round time to hour for pickup_datetime and dropoff_datetime
combined = combined.withColumn("pickup_hour", F.hour("pickup_datetime"))
combined = combined.withColumn("dropoff_hour", F.hour("dropoff_datetime"))

In [28]:
# extract date from pickup_datetime and dropoff_datetime
combined = combined.withColumn("pickup_date", F.to_date("pickup_datetime"))
combined = combined.withColumn("dropoff_date", F.to_date("dropoff_datetime"))

# drop pickup_datetime and dropoff_datetime
combined = combined.drop("pickup_datetime", "dropoff_datetime")

### Categorical Encoding

In [29]:
# One-hot encoding
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Indexing the categorical columns
vendor_indexer = StringIndexer(inputCol="VendorID", outputCol="VendorID_index")
ratecode_indexer = StringIndexer(inputCol="RatecodeID", outputCol="RatecodeID_index")
payment_type_indexer = StringIndexer(inputCol="payment_type", outputCol="payment_type_index")

# OneHotEncoding the indexed columns
vendor_encoder = OneHotEncoder(inputCol="VendorID_index", outputCol="VendorID_vec")
ratecode_encoder = OneHotEncoder(inputCol="RatecodeID_index", outputCol="RatecodeID_vec")
payment_type_encoder = OneHotEncoder(inputCol="payment_type_index", outputCol="payment_type_vec")

# Creating a pipeline to chain indexers and encoders
pipeline = Pipeline(stages=[vendor_indexer, ratecode_indexer, payment_type_indexer,
                            vendor_encoder, ratecode_encoder, payment_type_encoder])

# Fit the pipeline and transform the data
model = pipeline.fit(combined)
encoded_df = model.transform(combined)

# Drop the original columns after encoding
encoded_df = encoded_df.drop("VendorID", "RatecodeID", "payment_type")


## Export file to raw folder

In [33]:
# rearrange columns
tlc_df = encoded_df.select(
    "VendorID_vec",
    "pickup_date",
    "pickup_hour",
    "dropoff_date",
    "dropoff_hour",
    "trip_duration",
    "store_and_fwd_flag",
    "PULocationID",
    "DOLocationID",
    "passenger_count",
    "trip_distance",
    "RatecodeID_vec",
    "fare_amount",
    "extra",
    "mta_tax",
    "tip_amount",
    "tolls_amount",
    "improvement_surcharge",
    "congestion_surcharge",
    "ehail_fee", 
    "total_amount",
    "payment_type_vec"
)

In [34]:
# show the result
tlc_df.show(5)

+-------------+-----------+-----------+------------+------------+-------------+------------------+------------+------------+---------------+-------------+--------------+-----------+-----+-------+----------+------------+---------------------+--------------------+---------+------------+----------------+
| VendorID_vec|pickup_date|pickup_hour|dropoff_date|dropoff_hour|trip_duration|store_and_fwd_flag|PULocationID|DOLocationID|passenger_count|trip_distance|RatecodeID_vec|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|congestion_surcharge|ehail_fee|total_amount|payment_type_vec|
+-------------+-----------+-----------+------------+------------+-------------+------------------+------------+------------+---------------+-------------+--------------+-----------+-----+-------+----------+------------+---------------------+--------------------+---------+------------+----------------+
|(1,[0],[1.0])| 2023-12-01|          0|  2023-12-01|           0|          9.0|            

In [35]:
# export the combined data to parquet
combined.write.mode("overwrite").parquet("../data/raw/tlc_df.parquet")

24/08/27 19:44:48 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 148561 ms exceeds timeout 120000 ms
24/08/27 19:44:48 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/27 19:44:48 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$